In [1]:
import pandas as pd
import numpy as np

np.random.seed(1212)

import keras
from keras.models import Model
from keras.layers import *
from keras import optimizers

Using TensorFlow backend.


In [3]:
df_train = pd.read_csv('../input/train.csv')
df_test = pd.read_csv('../input/test.csv')

In [4]:


df_train.head() # 784 features, 1 label



,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
df_features = df_train.iloc[:, 1:785]
df_label = df_train.iloc[:, 0]

X_test = df_test.iloc[:, 0:784]

print(X_test.shape)

(28000, 784)


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split(df_features, df_label, 
                                                test_size = 0.2,
                                                random_state = 1212)

X_train = X_train.as_matrix().reshape(33600, 784) #(33600, 784)
X_cv = X_cv.as_matrix().reshape(8400, 784) #(8400, 784)

X_test = X_test.as_matrix().reshape(28000, 784)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if __name__ == '__main__':


In [7]:
print((min(X_train[1]), max(X_train[1])))

(0, 255)


In [8]:
# Feature Normalization 
X_train = X_train.astype('float32'); X_cv= X_cv.astype('float32'); X_test = X_test.astype('float32')
X_train /= 255; X_cv /= 255; X_test /= 255

# Convert labels to One Hot Encoded
num_digits = 10
y_train = keras.utils.to_categorical(y_train, num_digits)
y_cv = keras.utils.to_categorical(y_cv, num_digits)

In [9]:
# Printing 2 examples of labels after conversion
print(y_train[0]) # 2
print(y_train[3]) # 7

[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


In [10]:
# Input Parameters
n_input = 784 # number of features
n_hidden_1 = 300
n_hidden_2 = 100
n_hidden_3 = 100
n_hidden_4 = 200
num_digits = 10

In [11]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

In [12]:
# Our model would have '6' layers - input layer, 4 hidden layer and 1 output layer
model = Model(Inp, output)
model.summary() # We have 297,910 parameters to estimate

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 784)               0         
_________________________________________________________________
Hidden_Layer_1 (Dense)       (None, 300)               235500    
_________________________________________________________________
Hidden_Layer_2 (Dense)       (None, 100)               30100     
_________________________________________________________________
Hidden_Layer_3 (Dense)       (None, 100)               10100     
_________________________________________________________________
Hidden_Layer_4 (Dense)       (None, 200)               20200     
_________________________________________________________________
Output_Layer (Dense)         (None, 10)                2010      
Total params: 297,910
Trainable params: 297,910
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Insert Hyperparameters
learning_rate = 0.1
training_epochs = 20
batch_size = 100
sgd = optimizers.SGD(lr=learning_rate)

In [15]:
# We rely on the plain vanilla Stochastic Gradient Descent as our optimizing methodology
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [16]:
history1 = model.fit(X_train, y_train,
                     batch_size = batch_size,
                     epochs = training_epochs,
                     verbose = 2,
                     validation_data=(X_cv, y_cv))

Train on 33600 samples, validate on 8400 samples
Epoch 1/20
 - 4s - loss: 1.8541 - acc: 0.4985 - val_loss: 1.0047 - val_acc: 0.7601
Epoch 2/20
 - 2s - loss: 0.6482 - acc: 0.8293 - val_loss: 0.4641 - val_acc: 0.8720
Epoch 3/20
 - 2s - loss: 0.4099 - acc: 0.8834 - val_loss: 0.3621 - val_acc: 0.8975
Epoch 4/20
 - 2s - loss: 0.3377 - acc: 0.9027 - val_loss: 0.3123 - val_acc: 0.9104
Epoch 5/20
 - 2s - loss: 0.2981 - acc: 0.9138 - val_loss: 0.2892 - val_acc: 0.9175
Epoch 6/20
 - 2s - loss: 0.2684 - acc: 0.9227 - val_loss: 0.2652 - val_acc: 0.9243
Epoch 7/20
 - 2s - loss: 0.2454 - acc: 0.9297 - val_loss: 0.2557 - val_acc: 0.9258
Epoch 8/20
 - 2s - loss: 0.2273 - acc: 0.9351 - val_loss: 0.2321 - val_acc: 0.9336
Epoch 9/20
 - 2s - loss: 0.2102 - acc: 0.9377 - val_loss: 0.2175 - val_acc: 0.9363
Epoch 10/20
 - 2s - loss: 0.1952 - acc: 0.9440 - val_loss: 0.2053 - val_acc: 0.9394
Epoch 11/20
 - 2s - loss: 0.1828 - acc: 0.9469 - val_loss: 0.1953 - val_acc: 0.9427
Epoch 12/20
 - 2s - loss: 0.1708 - a

In [17]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

# We rely on ADAM as our optimizing methodology
adam = keras.optimizers.Adam(lr=learning_rate)
model2 = Model(Inp, output)

model2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [18]:
history2 = model2.fit(X_train, y_train,
                      batch_size = batch_size,
                      epochs = training_epochs,
                      verbose = 2,
                      validation_data=(X_cv, y_cv))

Train on 33600 samples, validate on 8400 samples
Epoch 1/20
 - 3s - loss: 0.3390 - acc: 0.8988 - val_loss: 0.1625 - val_acc: 0.9488
Epoch 2/20
 - 2s - loss: 0.1253 - acc: 0.9613 - val_loss: 0.1205 - val_acc: 0.9620
Epoch 3/20
 - 2s - loss: 0.0838 - acc: 0.9735 - val_loss: 0.1081 - val_acc: 0.9669
Epoch 4/20
 - 2s - loss: 0.0615 - acc: 0.9795 - val_loss: 0.0937 - val_acc: 0.9724
Epoch 5/20
 - 2s - loss: 0.0456 - acc: 0.9849 - val_loss: 0.1065 - val_acc: 0.9675
Epoch 6/20
 - 2s - loss: 0.0408 - acc: 0.9865 - val_loss: 0.0849 - val_acc: 0.9775
Epoch 7/20
 - 2s - loss: 0.0319 - acc: 0.9895 - val_loss: 0.1169 - val_acc: 0.9710
Epoch 8/20
 - 2s - loss: 0.0251 - acc: 0.9919 - val_loss: 0.0935 - val_acc: 0.9767
Epoch 9/20
 - 2s - loss: 0.0225 - acc: 0.9926 - val_loss: 0.1010 - val_acc: 0.9783
Epoch 10/20
 - 2s - loss: 0.0196 - acc: 0.9935 - val_loss: 0.1358 - val_acc: 0.9704
Epoch 11/20
 - 2s - loss: 0.0197 - acc: 0.9934 - val_loss: 0.1063 - val_acc: 0.9760
Epoch 12/20
 - 2s - loss: 0.0237 - a

In [19]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

learning_rate = 0.01
adam = keras.optimizers.Adam(lr=learning_rate)
model2a = Model(Inp, output)

model2a.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [20]:
history2a = model2a.fit(X_train, y_train,
                        batch_size = batch_size,
                        epochs = training_epochs,
                        verbose = 2,
                        validation_data=(X_cv, y_cv))

Train on 33600 samples, validate on 8400 samples
Epoch 1/20
 - 3s - loss: 0.3384 - acc: 0.8982 - val_loss: 0.1974 - val_acc: 0.9410
Epoch 2/20
 - 2s - loss: 0.1243 - acc: 0.9610 - val_loss: 0.1328 - val_acc: 0.9592
Epoch 3/20
 - 2s - loss: 0.0829 - acc: 0.9743 - val_loss: 0.1035 - val_acc: 0.9693
Epoch 4/20
 - 2s - loss: 0.0618 - acc: 0.9802 - val_loss: 0.0921 - val_acc: 0.9733
Epoch 5/20
 - 2s - loss: 0.0456 - acc: 0.9861 - val_loss: 0.1118 - val_acc: 0.9663
Epoch 6/20
 - 2s - loss: 0.0398 - acc: 0.9870 - val_loss: 0.0935 - val_acc: 0.9754
Epoch 7/20
 - 2s - loss: 0.0299 - acc: 0.9903 - val_loss: 0.1066 - val_acc: 0.9718
Epoch 8/20
 - 2s - loss: 0.0231 - acc: 0.9927 - val_loss: 0.1098 - val_acc: 0.9737
Epoch 9/20
 - 2s - loss: 0.0237 - acc: 0.9924 - val_loss: 0.1089 - val_acc: 0.9726
Epoch 10/20
 - 2s - loss: 0.0230 - acc: 0.9923 - val_loss: 0.1032 - val_acc: 0.9752
Epoch 11/20
 - 2s - loss: 0.0142 - acc: 0.9955 - val_loss: 0.0957 - val_acc: 0.9781
Epoch 12/20
 - 2s - loss: 0.0163 - a

In [21]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

learning_rate = 0.5
adam = keras.optimizers.Adam(lr=learning_rate)
model2b = Model(Inp, output)

model2b.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [22]:
history2b = model2b.fit(X_train, y_train,
                        batch_size = batch_size,
                        epochs = training_epochs,
                            validation_data=(X_cv, y_cv))

Train on 33600 samples, validate on 8400 samples
Epoch 1/20
33600/33600 [==============================] - 3s 88us/step - loss: 0.3335 - acc: 0.9026 - val_loss: 0.1468 - val_acc: 0.9577
Epoch 2/20
33600/33600 [==============================] - 3s 78us/step - loss: 0.1240 - acc: 0.9625 - val_loss: 0.1316 - val_acc: 0.9590
Epoch 3/20
33600/33600 [==============================] - 2s 72us/step - loss: 0.0799 - acc: 0.9747 - val_loss: 0.0995 - val_acc: 0.9699
Epoch 4/20
33600/33600 [==============================] - 2s 69us/step - loss: 0.0560 - acc: 0.9824 - val_loss: 0.0991 - val_acc: 0.9702
Epoch 5/20
33600/33600 [==============================] - 2s 68us/step - loss: 0.0502 - acc: 0.9840 - val_loss: 0.0835 - val_acc: 0.9760
Epoch 6/20
33600/33600 [==============================] - 2s 69us/step - loss: 0.0359 - acc: 0.9891 - val_loss: 0.0946 - val_acc: 0.9742
Epoch 7/20
33600/33600 [==============================] - 2s 69us/step - loss: 0.0260 - acc: 0.9916 - val_loss: 0.1196 - val_acc:

In [23]:
# Input Parameters
n_input = 784 # number of features
n_hidden_1 = 300
n_hidden_2 = 100
n_hidden_3 = 100
n_hidden_4 = 100
n_hidden_5 = 200
num_digits = 10

In [24]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
x = Dense(n_hidden_5, activation='relu', name = "Hidden_Layer_5")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

In [25]:
# Our model would have '7' layers - input layer, 5 hidden layer and 1 output layer
model3 = Model(Inp, output)
model3.summary() # We have 308,010 parameters to estimate

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 784)               0         
_________________________________________________________________
Hidden_Layer_1 (Dense)       (None, 300)               235500    
_________________________________________________________________
Hidden_Layer_2 (Dense)       (None, 100)               30100     
_________________________________________________________________
Hidden_Layer_3 (Dense)       (None, 100)               10100     
_________________________________________________________________
Hidden_Layer_4 (Dense)       (None, 100)               10100     
_________________________________________________________________
Hidden_Layer_5 (Dense)       (None, 200)               20200     
_________________________________________________________________
Output_Layer (Dense)         (None, 10)                2010      
Total para

In [26]:
# We rely on 'Adam' as our optimizing methodology
adam = keras.optimizers.Adam(lr=0.01)

model3.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [27]:
history3 = model3.fit(X_train, y_train,
                      batch_size = batch_size,
                      epochs = training_epochs,
                      validation_data=(X_cv, y_cv))

Train on 33600 samples, validate on 8400 samples
Epoch 1/20
33600/33600 [==============================] - 3s 95us/step - loss: 0.3594 - acc: 0.8936 - val_loss: 0.1994 - val_acc: 0.9415
Epoch 2/20
33600/33600 [==============================] - 2s 69us/step - loss: 0.1246 - acc: 0.9619 - val_loss: 0.1224 - val_acc: 0.9635
Epoch 3/20
33600/33600 [==============================] - 2s 69us/step - loss: 0.0834 - acc: 0.9737 - val_loss: 0.1134 - val_acc: 0.9662
Epoch 4/20
33600/33600 [==============================] - 2s 70us/step - loss: 0.0607 - acc: 0.9807 - val_loss: 0.1083 - val_acc: 0.9694
Epoch 5/20
33600/33600 [==============================] - 2s 70us/step - loss: 0.0494 - acc: 0.9846 - val_loss: 0.1186 - val_acc: 0.9671
Epoch 6/20
33600/33600 [==============================] - 2s 70us/step - loss: 0.0379 - acc: 0.9877 - val_loss: 0.0935 - val_acc: 0.9760
Epoch 7/20
33600/33600 [==============================] - 2s 70us/step - loss: 0.0361 - acc: 0.9878 - val_loss: 0.1099 - val_acc:

In [28]:
# Input Parameters
n_input = 784 # number of features
n_hidden_1 = 300
n_hidden_2 = 100
n_hidden_3 = 100
n_hidden_4 = 200
num_digits = 10

In [29]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dropout(0.3)(x)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dropout(0.3)(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dropout(0.3)(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

In [30]:
# Our model would have '6' layers - input layer, 4 hidden layer and 1 output layer
model4 = Model(Inp, output)
model4.summary() # We have 297,910 parameters to estimate

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 784)               0         
_________________________________________________________________
Hidden_Layer_1 (Dense)       (None, 300)               235500    
_________________________________________________________________
dropout_1 (Dropout)          (None, 300)               0         
_________________________________________________________________
Hidden_Layer_2 (Dense)       (None, 100)               30100     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
Hidden_Layer_3 (Dense)       (None, 100)               10100     
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
__________

In [31]:
model4.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [32]:
history = model4.fit(X_train, y_train,
                    batch_size = batch_size,
                    epochs = training_epochs,
                    validation_data=(X_cv, y_cv))

Train on 33600 samples, validate on 8400 samples
Epoch 1/20
33600/33600 [==============================] - 3s 97us/step - loss: 0.5818 - acc: 0.8125 - val_loss: 0.1879 - val_acc: 0.9415
Epoch 2/20
33600/33600 [==============================] - 2s 71us/step - loss: 0.2286 - acc: 0.9330 - val_loss: 0.1385 - val_acc: 0.9612
Epoch 3/20
33600/33600 [==============================] - 2s 71us/step - loss: 0.1759 - acc: 0.9482 - val_loss: 0.1227 - val_acc: 0.9644
Epoch 4/20
33600/33600 [==============================] - 2s 72us/step - loss: 0.1434 - acc: 0.9583 - val_loss: 0.1103 - val_acc: 0.9698
Epoch 5/20
33600/33600 [==============================] - 2s 71us/step - loss: 0.1235 - acc: 0.9642 - val_loss: 0.0952 - val_acc: 0.9732
Epoch 6/20
33600/33600 [==============================] - 2s 72us/step - loss: 0.1076 - acc: 0.9684 - val_loss: 0.0865 - val_acc: 0.9755
Epoch 7/20
33600/33600 [==============================] - 3s 84us/step - loss: 0.0994 - acc: 0.9699 - val_loss: 0.0978 - val_acc:

In [33]:
test_pred = pd.DataFrame(model4.predict(X_test, batch_size=200))
test_pred = pd.DataFrame(test_pred.idxmax(axis = 1))
test_pred.index.name = 'ImageId'
test_pred = test_pred.rename(columns = {0: 'Label'}).reset_index()
test_pred['ImageId'] = test_pred['ImageId'] + 1

test_pred.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3


In [34]:
test_pred.to_csv('mnist_submission.csv', index = False)

> Using this model, we are able to achieve a score of 0.976.